<a href="https://colab.research.google.com/github/shahinmomenzadeh/Company_attendance/blob/master/pflc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!mkdir /content/data

In [4]:
!wget -O /content/data/protein_links.txt "https://stringdb-static.org/download/protein.links.detailed.v11.5/9606.protein.links.detailed.v11.5.txt.gz"
!gzip -d /content/data/protein_links.txt

--2023-06-04 09:57:41--  https://stringdb-static.org/download/protein.links.detailed.v11.5/9606.protein.links.detailed.v11.5.txt.gz
Resolving stringdb-static.org (stringdb-static.org)... 151.106.34.104
Connecting to stringdb-static.org (stringdb-static.org)|151.106.34.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115547735 (110M) [application/octet-stream]
Saving to: ‘/content/data/protein_links.txt’

/content/data/prote 100%[===================>] 110.19M  15.6MB/s    in 7.5s    

2023-06-04 09:57:49 (14.8 MB/s) - ‘/content/data/protein_links.txt’ saved [115547735/115547735]

gzip: /content/data/protein_links.txt: unknown suffix -- ignored


In [6]:
!wget -O /content/data/protein_links.txt.gz "https://stringdb-static.org/download/protein.links.detailed.v11.5/9606.protein.links.detailed.v11.5.txt.gz"
!gzip -d /content/data/protein_links.txt.gz
dataset = pd.read_csv("/content/data/protein_links.txt", header=None, sep=" ")

--2023-06-04 10:00:17--  https://stringdb-static.org/download/protein.links.detailed.v11.5/9606.protein.links.detailed.v11.5.txt.gz
Resolving stringdb-static.org (stringdb-static.org)... 151.106.34.104
Connecting to stringdb-static.org (stringdb-static.org)|151.106.34.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115547735 (110M) [application/octet-stream]
Saving to: ‘/content/data/protein_links.txt.gz’

/content/data/prote 100%[===================>] 110.19M  15.6MB/s    in 7.5s    

2023-06-04 10:00:25 (14.7 MB/s) - ‘/content/data/protein_links.txt.gz’ saved [115547735/115547735]

gzip: /content/data/protein_links.txt already exists; do you wish to overwrite (y or n)? Y


<ipython-input-6-e9084f014579>:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("/content/data/protein_links.txt", header=None, sep=" ")


In [ ]:
import pandas as pd
import networkx as nx

# بارگیری دیتاست STRING
dataset = pd.read_csv("/content/data/protein_links.txt", sep=" ")

# فیلتر کردن ارتباطات با امتیاز بالا
df = dataset[dataset.combined_score >= 700]

# ساخت یک گراف از دیتاست STRING
G = nx.DiGraph()
for _, r in df.iterrows():
    G.add_node(r['protein1'], label=r['protein1'])
    G.add_node(r['protein2'], label=r['protein2'])
    G.add_edge(r['protein1'], r['protein2'], weight=r['combined_score'])

In [3]:
import numpy as np
# تبدیل گراف به گراف جهت‌دار
G = G.to_directed()

# محاسبه ماتریس لاپلاسی جهت‌دار
L = nx.directed_laplacian_matrix(G)
L = np.asarray(L)

In [ ]:
from numpy.linalg import eig

# محاسبه امتیازهای ویژه و بردارهای ویژه برای ماتریس لاپلاسی شبکه
eigenvalues, eigenvectors = eig(L)

# مرتب سازی امتیازهای ویژه به ترتیب صعودی
idx = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[idx]
eigenvectors = eigenvectors[:, idx]

# محاسبه امتیاز PFLC برای هر گره
PFLC = np.sum(np.abs(eigenvectors), axis=1)

In [ ]:
from sklearn.cluster import KMeans

# تعداد خوشه‌ها را مشخص کنید
n_clusters = 2

# اعمال الگوریتم خوشه‌بندی K-means بر روی امتیازهای PFLC
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(PFLC.reshape(-1, 1))

# شناسایی گروه‌های مرکزی بالا
high_central_nodes = np.where(kmeans.labels__ == kmeans.labels_.max())[0]